In [44]:
import pandas as pd
import numpy as np
import warnings
from google.cloud import bigquery
warnings.filterwarnings('ignore')
import requests
import time

In [7]:
# Se carga listado de pools uniswap v2, con información adicional de decimals, y conteo de eventos (NOTA: Se asumió decimals 18 para los tokens que no tenían información de decimals en la tabla de pools)
df_pool = pd.read_csv('/Users/gascalero/Documents/pipeline-rugpull/pipeline/data/raw/bq-results-20260214-162114-1771086081855.csv', delimiter=';')
df_pool.head()

,pair_address,token0,token1,creation_time,block_number,transaction_hash,token0_decimals,token1_decimals,sync_count,transfer_count,burn_count,mint_count
0,0x5a11b697767eab7c6948619f375491e2f7d0b99c,0x2c50ba1ed5e4574c1b613b044bd1876f0b0b87a9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2020-06-25 13:55:45 UTC,10335297,0xa1f67b6afbc96f37235d112684914ce0c12433cf0208...,18,18,2117,42,11,19
1,0xd80e0abf542c874e055da73f2a9d7ce492cdfb51,0x539efe69bcdd21a83efd9122571a64cc25e0282b,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2020-09-18 02:25:32 UTC,10883511,0x0b8f10a3e063a8b5d6c18b8d038e76e89c68a7a84de7...,8,18,1272,41,12,16
2,0x05948b660c30bd720bcab72ff40091660d8fc5e7,0x8e8fe3add2ab97dee73636d87d7cff113fd09746,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2020-12-20 10:23:39 UTC,11489676,0xc130bc7ee028d55f3c162bd647def6eae6c2cee8d719...,18,18,354,89,8,37
3,0x94a0ae2b443f9c0ca0fc6ae8a653a5c4e934b92e,0x36ac219f90f5a6a3c77f2a7b660e3cc701f68e25,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2020-11-26 22:51:16 UTC,11336836,0x5d3028beda99e3e8b994ab68d0b75109b7ae6e482518...,18,18,1009,59,18,22
4,0xa13c7131d0db3b9b1367ad68c9e67ad3ab8470c5,0x6ce2c24e6485213866aa7397c49ff31230097b25,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2021-02-11 10:10:57 UTC,11834649,0xc6168f759710e1454e1cd7f66b22b1668a58c0a0fc28...,18,18,505,106,8,42


In [21]:
left = df_pool[['token0','token0_decimals']].rename(columns={'token0':'token','token0_decimals':'decimals'})
right = df_pool[['token1','token1_decimals']].rename(columns={'token1':'token','token1_decimals':'decimals'})

pairs = pd.concat([left, right], ignore_index=True)
pairs['decimals'] = pairs['decimals'].astype(str)
pairs.drop_duplicates(inplace=True)

pairs.head()

,token,decimals
0,0x2c50ba1ed5e4574c1b613b044bd1876f0b0b87a9,18
1,0x539efe69bcdd21a83efd9122571a64cc25e0282b,8
2,0x8e8fe3add2ab97dee73636d87d7cff113fd09746,18
3,0x36ac219f90f5a6a3c77f2a7b660e3cc701f68e25,18
4,0x6ce2c24e6485213866aa7397c49ff31230097b25,18


# TOMAR MUESTRA DE TOKENS

**Muestreo Estratificado de Pools**

**Contexto**

Del universo de 37,087 pools (filtrados por wETH + >5 eventos Sync), se seleccionó una muestra de **998 tokens** mediante muestreo estratificado proporcional.

**Criterios de Estratificación**

La estratificación se basó en el nivel de actividad medido por `sync_count`:

- **Estrato Bajo** (6-26 Syncs): 504 tokens (50.5%)
- **Estrato Medio** (27-92 Syncs): 245 tokens (24.5%)  
- **Estrato Alto** (>92 Syncs): 249 tokens (25.0%)

**Justificación**

El muestreo proporcional replica la distribución natural observada en Uniswap V2, evitando sesgos artificiales que podrían afectar el entrenamiento del modelo y permitiendo comparación válida con los resultados de Mazorra et al.

**Reproducibilidad**

Se utilizó `random_state=42` para garantizar que el muestreo sea reproducible.

In [22]:
df_pool['sync_count'].describe()

count    3.708700e+04
mean     1.519535e+03
std      2.731549e+04
min      6.000000e+00
25%      1.200000e+01
50%      2.600000e+01
75%      9.200000e+01
max      3.600649e+06
Name: sync_count, dtype: float64

In [30]:
# 1. Crear columna de estrato
df_pool['estrato'] = pd.cut(df_pool['sync_count'], 
                        bins=[5, 26, 92, np.inf], 
                        labels=['bajo', 'medio', 'alto'])

# 2. Verificar distribución
print(df_pool['estrato'].value_counts())

# 3. Muestreo estratificado proporcional
muestra = df_pool.groupby('estrato', group_keys=False).apply(
    lambda x: x.sample(n=int(len(x) / len(df_pool) * 1000), random_state=42)
)

# 4. Verificar muestra
print(f"\nTotal muestra: {len(muestra)}")
print(muestra['estrato'].value_counts())

estrato
bajo     18721
alto      9268
medio     9098
Name: count, dtype: int64

Total muestra: 998
estrato
bajo     504
alto     249
medio    245
Name: count, dtype: int64


In [31]:
muestra.head()

,pair_address,token0,token1,creation_time,block_number,transaction_hash,token0_decimals,token1_decimals,sync_count,transfer_count,burn_count,mint_count,pair,estrato
25287,0x78a8805acf5d1a7b28a83d79bccda0def40b4335,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xebf919584021075d3f7bf3d6cf1c6dc318221eff,2020-09-29 10:58:50 UTC,10956992,0x1c8d02d0e4ff38e4483e59df48a7eba63da58b0fd0ce...,18,18,9,4,1,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2_0xe...,bajo
36139,0xa5ffe698f774ef96871a9641ff2b56b8ab97fc12,0x219865b49bea3a1638084cd1e8c6c87e36de308f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2021-06-07 07:31:28 UTC,12586083,0xc4e29f47011124d6056a10d204570dc310f44f8fd694...,18,18,9,11,2,6,0x219865b49bea3a1638084cd1e8c6c87e36de308f_0xc...,bajo
29796,0x7e47c5c0fe5838e9c29735923547b388bf95b7b7,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xfa235907a2705f3b55b222b29bb4637549fd3d28,2021-09-03 05:59:20 UTC,13150967,0xa1ecb4a0b2c382d388b9380fd3d107485a8ae67036ce...,18,18,14,6,1,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2_0xf...,bajo
18279,0xe5c1ad5f6f3f051cccebf9236e712dd1bf45b61a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xcccdb294b52fc00051bf694fc798efee33bc0358,2021-06-19 12:00:25 UTC,12664687,0x52ac61f074b1759f712540ac7afc64874c217c10a291...,18,18,13,4,1,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2_0xc...,bajo
34128,0xe89052e63a5914ed4fb5e5b7d507c36b8b7a4386,0x81835d805d3b4baeaf31655ee62fa6b7cafdb599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2021-05-03 00:47:32 UTC,12358068,0x0b688cefc175ac86961178bfdebab229c1bf542d29f4...,18,18,9,8,1,5,0x81835d805d3b4baeaf31655ee62fa6b7cafdb599_0xc...,bajo


# INGESTIÓN DE EVENTOS DE POOL (SYNC, MINT, BURN)

Se extraen todos los eventos Sync, Mint, Burn de la muestra de pools seleccionada. Previamente se garantizó que solo se seleccionaran pools con mínimo 5 eventos sync

In [38]:
# 1. Tu muestra ya está en memoria como 'muestra'
pair_addresses = muestra['pair_address'].tolist()
pair_list_str = "', '".join(pair_addresses)

# 2. Query de eventos del pool
query = f"""
-- SYNC EVENTS
SELECT 
  'sync' AS event_type,
  address AS pair_address,
  NULL AS sender,
  NULL AS to_address,
  CONCAT('0x', LOWER(SUBSTR(data, 3, 64))) AS amount0_or_reserve0_hex,
  CONCAT('0x', LOWER(SUBSTR(data, 67, 64))) AS amount1_or_reserve1_hex,
  block_timestamp,
  block_number,
  transaction_hash,
  log_index
FROM `bigquery-public-data.crypto_ethereum.logs`
WHERE topics[SAFE_OFFSET(0)] = '0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1'
  AND block_timestamp >= '2020-05-05 00:00:00 UTC'
  AND block_timestamp < '2021-09-14 00:00:00 UTC'
  AND address IN ('{pair_list_str}')

UNION ALL

-- MINT EVENTS
SELECT 
  'mint' AS event_type,
  address AS pair_address,
  CONCAT('0x', LOWER(SUBSTR(topics[SAFE_OFFSET(1)], 27, 40))) AS sender,
  NULL AS to_address,
  CONCAT('0x', LOWER(SUBSTR(data, 3, 64))) AS amount0_or_reserve0_hex,
  CONCAT('0x', LOWER(SUBSTR(data, 67, 64))) AS amount1_or_reserve1_hex,
  block_timestamp,
  block_number,
  transaction_hash,
  log_index
FROM `bigquery-public-data.crypto_ethereum.logs`
WHERE topics[SAFE_OFFSET(0)] = '0x4c209b5fc8ad50758f13e2e1088ba56a560dff690a1c6fef26394f4c03821c4f'
  AND block_timestamp >= '2020-05-05 00:00:00 UTC'
  AND block_timestamp < '2021-09-14 00:00:00 UTC'
  AND address IN ('{pair_list_str}')

UNION ALL

-- BURN EVENTS
SELECT 
  'burn' AS event_type,
  address AS pair_address,
  CONCAT('0x', LOWER(SUBSTR(topics[SAFE_OFFSET(1)], 27, 40))) AS sender,
  CONCAT('0x', LOWER(SUBSTR(topics[SAFE_OFFSET(2)], 27, 40))) AS to_address,
  CONCAT('0x', LOWER(SUBSTR(data, 3, 64))) AS amount0_or_reserve0_hex,
  CONCAT('0x', LOWER(SUBSTR(data, 67, 64))) AS amount1_or_reserve1_hex,
  block_timestamp,
  block_number,
  transaction_hash,
  log_index
FROM `bigquery-public-data.crypto_ethereum.logs`
WHERE topics[SAFE_OFFSET(0)] = '0xdccd412f0b1252819cb1fd330b93224ca42612892bb3f4f789976e6d81936496'
  AND block_timestamp >= '2020-05-05 00:00:00 UTC'
  AND block_timestamp < '2021-09-14 00:00:00 UTC'
  AND address IN ('{pair_list_str}')

ORDER BY block_timestamp, pair_address, log_index
"""

# 3. Ejecutar query
client = bigquery.Client(project='uniswap-explore-475916')  # Ajusta tu proyecto
df_eventos_pool = client.query(query).to_dataframe()

# 4. Guardar
df_eventos_pool.to_csv('/Users/gascalero/Documents/pipeline-rugpull/pipeline/data/raw/eventos_pool_sync_mint_burn.csv', index=False)

print(f"Total eventos: {len(df_eventos_pool)}")
print(df_eventos_pool['event_type'].value_counts())

Total eventos: 1835994
event_type
sync    1755374
mint      51078
burn      29542
Name: count, dtype: int64


In [40]:
print(df_eventos_pool['event_type'].value_counts())

event_type
sync    1755374
mint      51078
burn      29542
Name: count, dtype: int64


# INGESTIÓN DE EVENTOS TRANSFER

In [41]:
# 1. Identificar tokens NO-wETH de tu muestra
weth = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'

muestra['token_address'] = muestra.apply(
    lambda row: row['token1'] if row['token0'] == weth else row['token0'],
    axis=1
)

# 2. Lista de tokens únicos (sin wETH)
tokens = muestra['token_address'].unique().tolist()
print(f"Total tokens únicos a extraer: {len(tokens)}")

# 3. Generar string para el query
token_list_str = "', '".join(tokens)

# 4. Query de Transfer events
query = f"""
SELECT 
    token_address,
    from_address,
    to_address,
    value,
    transaction_hash,
    block_timestamp,
    block_number,
    log_index
FROM `bigquery-public-data.crypto_ethereum.token_transfers`
WHERE block_timestamp >= '2020-05-05 00:00:00 UTC'
  AND block_timestamp < '2021-09-14 00:00:00 UTC'
  AND token_address IN ('{token_list_str}')
ORDER BY block_timestamp, token_address, log_index
"""

# 5. Ejecutar query
client = bigquery.Client(project='uniswap-explore-475916')  # Ajusta tu proyecto
print("Ejecutando query de Transfer events...")
df_transfers = client.query(query).to_dataframe()

# 6. Guardar resultados
df_transfers.to_csv('/Users/gascalero/Documents/pipeline-rugpull/pipeline/data/raw/eventos_transfers_tokens.csv', index=False)

print(f"Total Transfer events extraídos: {len(df_transfers):,}")
print(f"\nPrimeras filas:")
print(df_transfers.head())

Total tokens únicos a extraer: 998
Ejecutando query de Transfer events...
Total Transfer events extraídos: 8,717,043

Primeras filas:
                                token_address  \
0  0xdd974d5c2e2928dea5f71b9825b8b646686bd200   
1  0xdd974d5c2e2928dea5f71b9825b8b646686bd200   
2  0xdd974d5c2e2928dea5f71b9825b8b646686bd200   
3  0x2260fac5e5542a773aa44fbcfedf7c193bc2c599   
4  0x2260fac5e5542a773aa44fbcfedf7c193bc2c599   

                                 from_address  \
0  0x794973d99724d14025d41a767360c2aae21d2c37   
1  0x65bf64ff5f51272f729bdcd7acfb00677ced86cd   
2  0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be   
3  0x794e6e91555438afc3ccf1c5076a74f42133d08d   
4  0x38caa9a43c911a5bd745c5ba8d40dbaad26f90e1   

                                   to_address                   value  \
0  0x65bf64ff5f51272f729bdcd7acfb00677ced86cd  4998000000000000524288   
1  0x607d7751d9f4845c5a1de9eed39c56f4fc0f855d  4998000000000000524288   
2  0xd6e8f8c4fc6fce1d77fd01f7749ae3f35f425d68  2502066100

In [42]:
print(f"Total Transfer events: {len(df_transfers):,}")

# Distribución por token (top 10 más activos)
print("\nTop 10 tokens con más transfers:")
print(df_transfers['token_address'].value_counts().head(10))

Total Transfer events: 8,717,043

Top 10 tokens con más transfers:
token_address
0x0000000000004946c0e9f43f4dee607b0ef1fa1c    2546400
0x2260fac5e5542a773aa44fbcfedf7c193bc2c599    2415182
0xdd974d5c2e2928dea5f71b9825b8b646686bd200     720154
0xcadb96858fe496bb6309622f9023ba2defb5d540     381139
0xf938424f7210f31df2aee3011291b658f872e91e     218731
0xa80f2c8f61c56546001f5fc2eb8d6e4e72c45d4c     216074
0xc40af1e4fecfa05ce6bab79dcd8b373d2e436c4e     186345
0x9992ec3cf6a55b00978cddf2b27bc6882d88d1ec     153641
0x4c11249814f11b9346808179cf06e71ac328c1b5     151051
0xe0ad1806fd3e7edf6ff52fdb822432e847411033     112355
Name: count, dtype: int64


# INGESTIÓN TOKEN CREATOR ADDRESS, TRANSACTION HASH, BLOCK Y TIMESTAMP (MÁS COMPLETO QUE EL ANTERIOR)

In [ ]:
# 1. Tokens únicos
tokens = muestra['token_address'].unique().tolist()
print(f"Total tokens a consultar: {len(tokens)}")

# 2. API key
api_key = 'TEP6AB68TS7R51RBQUXEU47W55A3FIXZEM'
results = []

print("Consultando metadata completa de tokens en Etherscan...")

for i, token in enumerate(tokens):
    # Paso 1: Obtener contract creation
    url_creation = f"https://api.etherscan.io/v2/api?chainid=1&module=contract&action=getcontractcreation&contractaddresses={token}&apikey={api_key}"
    
    try:
        response = requests.get(url_creation)
        data = response.json()
        
        if data['status'] == '1' and data['result']:
            creator_info = data['result'][0]
            tx_hash = creator_info['txHash']
            creator = creator_info['contractCreator']
            
            time.sleep(0.34)  # Rate limit
            
            # Paso 2: Obtener detalles de la transacción
            url_tx = f"https://api.etherscan.io/v2/api?chainid=1&module=proxy&action=eth_getTransactionByHash&txhash={tx_hash}&apikey={api_key}"
            response_tx = requests.get(url_tx)
            data_tx = response_tx.json()
            
            if data_tx['result']:
                block_number = int(data_tx['result']['blockNumber'], 16)
                
                time.sleep(0.34)  # Rate limit
                
                # Paso 3: Obtener timestamp del bloque
                block_hex = hex(block_number)
                url_block = f"https://api.etherscan.io/v2/api?chainid=1&module=proxy&action=eth_getBlockByNumber&tag={block_hex}&boolean=false&apikey={api_key}"
                response_block = requests.get(url_block)
                data_block = response_block.json()
                
                if data_block['result']:
                    timestamp = int(data_block['result']['timestamp'], 16)
                    
                    results.append({
                        'token_address': token,
                        'token_creator': creator,
                        'token_creation_tx': tx_hash,
                        'token_creation_block': block_number,
                        'token_creation_timestamp': pd.to_datetime(timestamp, unit='s')
                    })
                else:
                    results.append({'token_address': token, 'token_creator': creator, 'token_creation_tx': tx_hash, 'token_creation_block': block_number, 'token_creation_timestamp': None})
            else:
                results.append({'token_address': token, 'token_creator': creator, 'token_creation_tx': tx_hash, 'token_creation_block': None, 'token_creation_timestamp': None})
        else:
            results.append({'token_address': token, 'token_creator': None, 'token_creation_tx': None, 'token_creation_block': None, 'token_creation_timestamp': None})
    
    except Exception as e:
        print(f"Error en token {token}: {e}")
        results.append({'token_address': token, 'token_creator': None, 'token_creation_tx': None, 'token_creation_block': None, 'token_creation_timestamp': None})
    
    if (i + 1) % 50 == 0:
        print(f"Procesados {i + 1}/{len(tokens)} tokens...")

# Crear DataFrame final
df_token_metadata = pd.DataFrame(results)
df_token_metadata.to_csv('/Users/gascalero/Documents/pipeline-rugpull/pipeline/data/raw/token_metadata_complete.csv', index=False)

print(f"\n✅ Completado!")
print(df_token_metadata.head())

Total tokens a consultar: 998
Consultando metadata completa de tokens en Etherscan...
Procesados 50/998 tokens...
Procesados 100/998 tokens...
Procesados 150/998 tokens...
Procesados 200/998 tokens...
Procesados 250/998 tokens...
Procesados 300/998 tokens...
Procesados 350/998 tokens...
Procesados 400/998 tokens...
Procesados 450/998 tokens...
Procesados 500/998 tokens...
Procesados 550/998 tokens...
Procesados 600/998 tokens...
Procesados 650/998 tokens...
Procesados 700/998 tokens...
Procesados 750/998 tokens...
Procesados 800/998 tokens...
Procesados 850/998 tokens...
Procesados 900/998 tokens...
Procesados 950/998 tokens...

✅ Completado!
                                token_address  \
0  0xebf919584021075d3f7bf3d6cf1c6dc318221eff   
1  0x219865b49bea3a1638084cd1e8c6c87e36de308f   
2  0xfa235907a2705f3b55b222b29bb4637549fd3d28   
3  0xcccdb294b52fc00051bf694fc798efee33bc0358   
4  0x81835d805d3b4baeaf31655ee62fa6b7cafdb599   

                                token_creator  \
0  0x

In [54]:
df_token_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   token_address             998 non-null    object        
 1   token_creator             998 non-null    object        
 2   token_creation_tx         998 non-null    object        
 3   token_creation_block      998 non-null    int64         
 4   token_creation_timestamp  998 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 39.1+ KB


# POOL CREATOR ADDRESS

In [ ]:
# 1. Lista de transaction hashes de pools
pool_tx_hashes = muestra['transaction_hash'].tolist()
pool_tx_list_str = "', '".join(pool_tx_hashes)

# 2. Query
query = f"""
SELECT 
  `hash` AS transaction_hash,
  from_address AS pool_creator
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE `hash` IN ('{pool_tx_list_str}')
"""

# 3. Ejecutar
client = bigquery.Client(project='uniswap-explore-475916')
df_pool_creators = client.query(query).to_dataframe()

# 4. Merge
muestra = muestra.merge(df_pool_creators, on='transaction_hash', how='left')

print(f"✅ Pools con creator: {muestra['pool_creator'].notna().sum()}/998")
muestra.to_csv('/Users/gascalero/Documents/pipeline-rugpull/pipeline/data/raw/pool_list_complete.csv', index=False)

✅ Pools con creator: 998/998


In [58]:
muestra.head()

,pair_address,token0,token1,creation_time,block_number,transaction_hash,token0_decimals,token1_decimals,sync_count,transfer_count,burn_count,mint_count,pair,estrato,token_address,pool_creator
0,0x78a8805acf5d1a7b28a83d79bccda0def40b4335,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xebf919584021075d3f7bf3d6cf1c6dc318221eff,2020-09-29 10:58:50 UTC,10956992,0x1c8d02d0e4ff38e4483e59df48a7eba63da58b0fd0ce...,18,18,9,4,1,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2_0xe...,bajo,0xebf919584021075d3f7bf3d6cf1c6dc318221eff,0x70482e14b07a286dc3c61650e37fd4c33e3273dc
1,0xa5ffe698f774ef96871a9641ff2b56b8ab97fc12,0x219865b49bea3a1638084cd1e8c6c87e36de308f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2021-06-07 07:31:28 UTC,12586083,0xc4e29f47011124d6056a10d204570dc310f44f8fd694...,18,18,9,11,2,6,0x219865b49bea3a1638084cd1e8c6c87e36de308f_0xc...,bajo,0x219865b49bea3a1638084cd1e8c6c87e36de308f,0x2280ef90cacea9f5275f925736d840a6580a912c
2,0x7e47c5c0fe5838e9c29735923547b388bf95b7b7,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xfa235907a2705f3b55b222b29bb4637549fd3d28,2021-09-03 05:59:20 UTC,13150967,0xa1ecb4a0b2c382d388b9380fd3d107485a8ae67036ce...,18,18,14,6,1,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2_0xf...,bajo,0xfa235907a2705f3b55b222b29bb4637549fd3d28,0x30d1622155729a2091e9546b97f05cd1ea1c17fb
3,0xe5c1ad5f6f3f051cccebf9236e712dd1bf45b61a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xcccdb294b52fc00051bf694fc798efee33bc0358,2021-06-19 12:00:25 UTC,12664687,0x52ac61f074b1759f712540ac7afc64874c217c10a291...,18,18,13,4,1,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2_0xc...,bajo,0xcccdb294b52fc00051bf694fc798efee33bc0358,0xc184402dfcc31807dcfa92462a371382159870f3
4,0xe89052e63a5914ed4fb5e5b7d507c36b8b7a4386,0x81835d805d3b4baeaf31655ee62fa6b7cafdb599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2021-05-03 00:47:32 UTC,12358068,0x0b688cefc175ac86961178bfdebab229c1bf542d29f4...,18,18,9,8,1,5,0x81835d805d3b4baeaf31655ee62fa6b7cafdb599_0xc...,bajo,0x81835d805d3b4baeaf31655ee62fa6b7cafdb599,0xbb194ff5000c9d04818f59c7bf8be564d4dd5f0c
